<h1>导入数据</h1>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import time
import tensorflow.keras.backend as K
import keras_metrics as km
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from tensorflow.keras.layers import Input, Dense, LSTM, MaxPool1D, Dropout, concatenate, Flatten, Conv1D, Lambda, BatchNormalization, ReLU, Activation
from tensorflow.keras.callbacks import History, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import Hinge
from tensorflow.keras.metrics import Precision, Recall
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.backend import clear_session, epsilon
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, save_model, load_model
from time import strftime, localtime
from itertools import combinations

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [2]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [3]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [4]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [5]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [6]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [7]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [8]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [9]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [10]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0,0,0,0,0,0,1,0,1,1


In [11]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                8840 non-null   float64
 1   RUN_CAP                     8840 non-null   float64
 2   SHIFT_NO                    8840 non-null   float64
 3   CHK_CYCLE                   8840 non-null   float64
 4   IS_FLAG                     8840 non-null   int64  
 5   live_days                   8840 non-null   int64  
 6   check_days                  8840 non-null   int64  
 7   ELEC_TYPE_NAME_乡村居民生活用电     8840 non-null   uint8  
 8   ELEC_TYPE_NAME_商业用电         8840 non-null   uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     8840 non-null   uint8  
 10  ELEC_TYPE_NAME_学校教学和学生生活用电  8840 non-null   uint8  
 11  ELEC_TYPE_NAME_居民生活用电       8840 non-null   uint8  
 12  ELEC_TYPE_NAME_普通工业         8840 non-null   uint8  
 13  ELEC_TYPE_NAME_非居民照

<h3>elec_month数据预处理</h3>

In [12]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [13]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] = elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] / elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]].max(axis=0)
elec_month

In [14]:
cons_month_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_month_result.append(df.values)
cons_month = np.array(cons_month_result)
cons_month.shape

100%|███████████████████████████████████████████████████████████████████████████████| 8840/8840 [00:13<00:00, 665.46it/s]


(8840, 22, 4)

<h3>elec_day数据预处理</h3>

In [15]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] = elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] / elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]].max(axis=0)
elec_day

In [16]:
dates = pd.to_datetime(elec_day["rq"].unique()).sort_values()

In [17]:
cons_day_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id].drop(columns=["id"]).sort_values("rq")
    df["rq"] = pd.to_datetime(df["rq"])
    df = df.set_index("rq").reindex(dates)
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_day_result.append(df.values)
cons_day = np.array(cons_day_result)
cons_day.shape

100%|███████████████████████████████████████████████████████████████████████████████| 8840/8840 [00:25<00:00, 342.49it/s]


(8840, 108, 6)

<h2>再处理数据</h2>

<h3>查看现有数据形状</h3>

<h4>标签</h4>

In [18]:
target = cons_info["IS_FLAG"].values.astype(np.float64)
target.shape

(8840,)

<h4>统一视图</h4>

In [19]:
cons_info_train = cons_info.drop(columns=["IS_FLAG"]).values
print(cons_info_train.shape)
print(np.isnan(cons_info_train).sum())

(8840, 20)
0


<h4>月均电量</h4>

In [20]:
print(cons_month.shape)
print(np.isnan(cons_month).sum())

(8840, 22, 4)
0


<h4>日电量</h4>

In [21]:
print(cons_day.shape)
print(np.isnan(cons_day).sum())

(8840, 108, 6)
0


<h3>处理样本平衡性</h3>

In [22]:
pos = target == 1
neg = target == 0
repeat_num = 800

target = np.concatenate([target[pos].repeat(repeat_num, axis=0), target[neg]], axis=0)
target.shape

cons_info_train = np.concatenate([cons_info_train[pos].repeat(repeat_num, axis=0), cons_info_train[neg]], axis=0)
cons_info_train.shape

cons_month = np.concatenate([cons_month[pos].repeat(repeat_num, axis=0), cons_month[neg]], axis=0)
cons_month.shape

cons_day = np.concatenate([cons_day[pos].repeat(repeat_num, axis=0), cons_day[neg]], axis=0)
cons_day.shape

<h1>构建模型</h1>

<h2>初始化tensorflow</h2>

In [23]:
clear_session()
tf.compat.v1.reset_default_graph()
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h2>构建模型</h2>

In [24]:
def add_features(x, columns: [int]):
    """
    增加新的特征\n
    """
    if len(columns) == 0:
        return x
    for i, j in combinations(columns, r=2):
        feature = tf.expand_dims(x[:, :, i] - x[:, :, j], -1)
        x = tf.concat([x, feature], axis=-1)
    return x

In [25]:
def build_model(cons_info_shape, cons_month_shape, cons_day_shape) -> Model:
    input_cons_info = Input(shape=cons_info_shape, name="input_cons_info")
    input_cons_month = Input(shape=cons_month_shape, name="input_cons_month")
    input_cons_day = Input(shape=cons_day_shape, name="input_cons_day")
    
    x0 = BatchNormalization(name="x0")(input_cons_info)
    x1 = Dense(units=32, activation="relu", name="x1")(x0)
    x2 = Dense(units=64, activation="relu", name="x2")(x1)
    x3 = Dense(units=128, activation=None, name="x3")(x2)
    x4 = BatchNormalization(name="x4")(x3)
    x5 = ReLU(name="x5")(x4)
    
#     y11 = Conv1D(filters=16, kernel_size=5, strides=1, padding="causal", activation="relu", name="conv1d_y1")(input_cons_month)
#     y12 = MaxPool1D(pool_size=5, strides=1, padding="valid", name="max_pool1d_y1")(y11)
#     y13 = Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", activation="relu", name="conv1d_y2")(y12)
#     y14 = MaxPool1D(pool_size=3, strides=1, padding="valid", name="max_pool1d_y2")(y13)
    y14 = Lambda(lambda x: add_features(x, [0, 1, 2]), name="lambda_y")(input_cons_month)
    y1, y_hidden_state, y_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="y1")(y14)
    y_hidden_state_1 = Dense(units=64, name="y_hidden_state_1")(y_hidden_state)
    y_cell_state_1 = Dense(units=64, name="y_cell_state_1")(y_cell_state)
    y2 = LSTM(units=64, return_sequences=True, return_state=False, name="y2")(y1, [y_hidden_state_1, y_cell_state_1])
    
#     z11 = Conv1D(filters=16, kernel_size=5, strides=1, padding="causal", activation="relu", name="conv1d_z1")(input_cons_day)
#     z12 = MaxPool1D(pool_size=5, strides=1, padding="valid", name="max_pool1d_z1")(z11)
#     z13 = Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", activation="relu", name="conv1d_z2")(z12)
#     z14 = MaxPool1D(pool_size=3, strides=1, padding="valid", name="max_pool1d_z2")(z13)
    z14 = Lambda(lambda x: add_features(x, [3, 4, 5]), name="lambda_z")(input_cons_day)
    z1, z_hidden_state, z_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="z1")(z14)
    z_hidden_state_1 = Dense(units=64, name="z_hidden_state_1")(z_hidden_state)
    z_cell_state_1 = Dense(units=64, name="z_cell_state_1")(z_cell_state)
    z2 = LSTM(units=64, return_sequences=True, return_state=False, name="z2")(z1, [z_hidden_state_1, z_cell_state_1])
    
    s1 = concatenate([y2, z2], axis=1, name="s1")
    s2 = LSTM(units=16, return_sequences=True, return_state=False, name="s2")(s1)
    s3 = LSTM(units=8, return_sequences=True, return_state=False, name="s3")(s2)
    s4 = Flatten(name="s4")(s3)
    
    o1 = concatenate([x5, s4], axis=-1, name="o1")
    o2 = ReLU(name="relu_o2")(BatchNormalization(name="bn_o2")(Dense(units=64, activation=None, name="o2")(o1)))
    o3 = ReLU(name="relu_o3")(BatchNormalization(name="bn_o3")(Dense(units=128, activation=None, name="o3")(o2)))
    o4 = ReLU(name="relu_o4")(BatchNormalization(name="bn_o4")(Dense(units=64, activation=None, name="o4")(o3)))
    o5 = ReLU(name="relu_o5")(BatchNormalization(name="bn_o5")(Dense(units=32, activation=None, name="o5")(o4)))
    o6 = ReLU(name="relu_o6")(BatchNormalization(name="bn_o6")(Dense(units=16, activation=None, name="o6")(o5)))
    o7 = Activation(activation="sigmoid", name="relu_o7")(BatchNormalization(name="bn_o7")(Dense(units=1, activation=None, name="o7")(o6)))
    
    result = Model(inputs=[input_cons_info, input_cons_month, input_cons_day], outputs=[o7], name="mining-detector")
    result.summary()
    return result

In [26]:
def f1_loss(y_true, y_pred):
    #计算tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    
    #percision与recall，这里的K.epsilon代表一个小正数，用来避免分母为零
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    #计算f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)#其实就是把nan换成0
    return 1 - K.mean(f1)

In [27]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [28]:
model = build_model(cons_info_train.shape[1:], cons_month.shape[1:], cons_day.shape[1:])
model.compile(
    optimizer=Adam(learning_rate=1e-4), 
    loss=f1_loss, 
    metrics=[
        "accuracy", 
        F1Score(num_classes=1, average='weighted',threshold=0.5), 
        Precision(thresholds=0.5), 
        Recall(thresholds=0.5)
    ]
)

Model: "mining-detector"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cons_month (InputLayer)   [(None, 22, 4)]      0                                            
__________________________________________________________________________________________________
input_cons_day (InputLayer)     [(None, 108, 6)]     0                                            
__________________________________________________________________________________________________
lambda_y (Lambda)               (None, 22, 7)        0           input_cons_month[0][0]           
__________________________________________________________________________________________________
lambda_z (Lambda)               (None, 108, 9)       0           input_cons_day[0][0]             
____________________________________________________________________________________

<h2>训练模型</h2>

<h3>设置class_weight和sample_weight</h3>

In [29]:
class_weight = compute_class_weight('balanced', np.unique(target) ,target)
class_weights = dict()
for class_name, weight in enumerate(class_weight):
    class_weights[class_name] = weight
    
sample_weights = np.ones(shape=(len(target),))
sample_weights[target == 1] = 800

c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


<h3>开始训练</h3>

In [30]:
model.fit(
    [cons_info_train, cons_month, cons_day], 
    target, 
#     validation_split=0.2,
    epochs=50000, 
    batch_size=1024, 
#     class_weight=class_weights,
    use_multiprocessing=True, 
    workers=12,
    callbacks=[
        TensorBoard(log_dir="../logs/%s" % strftime("%Y%m%d_%H.%M.%S", localtime()), write_graph=True, write_images=True),
        ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5, min_lr=1e-6),
#         EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, restore_best_weights=True)
    ]
)

Epoch 1/50000
1/9 [==>...........................] - ETA: 0s - loss: 0.9981 - accuracy: 0.6230 - f1_score: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00WARNING:tensorflow:From c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
9/9 [==============================] - 1s 82ms/step - loss: 0.9976 - accuracy: 0.6232 - f1_score: 0.0024 - precision: 0.0012 - recall: 0.4000
Epoch 2/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9973 - accuracy: 0.6490 - f1_score: 0.0039 - precision: 0.0019 - recall: 0.6000
Epoch 3/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9967 - accuracy: 0.6581 - f1_score: 0.0059 - precision: 0.0030 - recall: 0.9000
Epoch 4/50000
9/9 [==============================] - 0s 39ms/step - loss:

9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.7933 - f1_score: 0.0108 - precision: 0.0054 - recall: 1.0000
Epoch 49/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8017 - f1_score: 0.0113 - precision: 0.0057 - recall: 1.0000
Epoch 50/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8045 - f1_score: 0.0114 - precision: 0.0058 - recall: 1.0000
Epoch 51/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8027 - f1_score: 0.0113 - precision: 0.0057 - recall: 1.0000
Epoch 52/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8002 - f1_score: 0.0112 - precision: 0.0056 - recall: 1.0000
Epoch 53/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8042 - f1_score: 0.0114 - precision: 0.0057 - recall: 1.0000
Epoch 54/50000
9/9 [==============================] - 0s 39ms/step - loss

9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.7998 - f1_score: 0.0112 - precision: 0.0056 - recall: 1.0000
Epoch 100/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.7959 - f1_score: 0.0110 - precision: 0.0055 - recall: 1.0000
Epoch 101/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8040 - f1_score: 0.0114 - precision: 0.0057 - recall: 1.0000
Epoch 102/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8043 - f1_score: 0.0114 - precision: 0.0057 - recall: 1.0000
Epoch 103/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8028 - f1_score: 0.0113 - precision: 0.0057 - recall: 1.0000
Epoch 104/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8038 - f1_score: 0.0114 - precision: 0.0057 - recall: 1.0000
Epoch 105/50000
9/9 [==============================] - 0s 39ms/step 

9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8031 - f1_score: 0.0114 - precision: 0.0057 - recall: 1.0000
Epoch 151/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8068 - f1_score: 0.0116 - precision: 0.0058 - recall: 1.0000
Epoch 152/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9952 - accuracy: 0.8090 - f1_score: 0.0117 - precision: 0.0059 - recall: 1.0000
Epoch 153/50000
9/9 [==============================] - 0s 46ms/step - loss: 0.9952 - accuracy: 0.8100 - f1_score: 0.0118 - precision: 0.0059 - recall: 1.0000
Epoch 154/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8088 - f1_score: 0.0117 - precision: 0.0059 - recall: 1.0000
Epoch 155/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8090 - f1_score: 0.0117 - precision: 0.0059 - recall: 1.0000
Epoch 156/50000
9/9 [==============================] - 0s 39ms/step 

9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8121 - f1_score: 0.0119 - precision: 0.0060 - recall: 1.0000
Epoch 202/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8119 - f1_score: 0.0119 - precision: 0.0060 - recall: 1.0000
Epoch 203/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8075 - f1_score: 0.0116 - precision: 0.0058 - recall: 1.0000
Epoch 204/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8192 - f1_score: 0.0124 - precision: 0.0062 - recall: 1.0000
Epoch 205/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8123 - f1_score: 0.0119 - precision: 0.0060 - recall: 1.0000
Epoch 206/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9952 - accuracy: 0.8139 - f1_score: 0.0120 - precision: 0.0060 - recall: 1.0000
Epoch 207/50000
9/9 [==============================] - 0s 39ms/step 

...

9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9325 - f1_score: 0.0324 - precision: 0.0165 - recall: 1.0000
Epoch 4278/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9319 - f1_score: 0.0322 - precision: 0.0163 - recall: 1.0000
Epoch 4279/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9318 - f1_score: 0.0321 - precision: 0.0163 - recall: 1.0000 
Epoch 4280/50000
9/9 [==============================] - 0s 41ms/step - loss: 0.9951 - accuracy: 0.9317 - f1_score: 0.0321 - precision: 0.0163 - recall: 1.0000
Epoch 4281/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9318 - f1_score: 0.0321 - precision: 0.0163 - recall: 1.0000
Epoch 4282/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9311 - f1_score: 0.0318 - precision: 0.0162 - recall: 1.0000
Epoch 4283/50000
9/9 [==============================] - 0s 40m

9/9 [==============================] - 0s 41ms/step - loss: 0.9951 - accuracy: 0.9327 - f1_score: 0.0325 - precision: 0.0165 - recall: 1.0000
Epoch 4329/50000
9/9 [==============================] - 0s 42ms/step - loss: 0.9951 - accuracy: 0.9325 - f1_score: 0.0324 - precision: 0.0165 - recall: 1.0000
Epoch 4330/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9319 - f1_score: 0.0322 - precision: 0.0163 - recall: 1.0000
Epoch 4331/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9309 - f1_score: 0.0317 - precision: 0.0161 - recall: 1.0000
Epoch 4332/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9324 - f1_score: 0.0324 - precision: 0.0164 - recall: 1.0000
Epoch 4333/50000
9/9 [==============================] - 0s 41ms/step - loss: 0.9951 - accuracy: 0.9322 - f1_score: 0.0323 - precision: 0.0164 - recall: 1.0000
Epoch 4334/50000
9/9 [==============================] - 0s 40ms

9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9309 - f1_score: 0.0317 - precision: 0.0161 - recall: 1.0000
Epoch 4380/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9322 - f1_score: 0.0323 - precision: 0.0164 - recall: 1.0000
Epoch 4381/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9309 - f1_score: 0.0317 - precision: 0.0161 - recall: 1.0000
Epoch 4382/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9309 - f1_score: 0.0317 - precision: 0.0161 - recall: 1.0000
Epoch 4383/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9326 - f1_score: 0.0325 - precision: 0.0165 - recall: 1.0000
Epoch 4384/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9317 - f1_score: 0.0321 - precision: 0.0163 - recall: 1.0000
Epoch 4385/50000
9/9 [==============================] - 0s 40ms

9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9316 - f1_score: 0.0320 - precision: 0.0163 - recall: 1.0000
Epoch 4430/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9301 - f1_score: 0.0313 - precision: 0.0159 - recall: 1.0000
Epoch 4431/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9311 - f1_score: 0.0318 - precision: 0.0162 - recall: 1.0000
Epoch 4432/50000
9/9 [==============================] - 0s 39ms/step - loss: 0.9951 - accuracy: 0.9327 - f1_score: 0.0325 - precision: 0.0165 - recall: 1.0000
Epoch 4433/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9328 - f1_score: 0.0326 - precision: 0.0166 - recall: 1.0000
Epoch 4434/50000
9/9 [==============================] - 0s 40ms/step - loss: 0.9951 - accuracy: 0.9311 - f1_score: 0.0318 - precision: 0.0162 - recall: 1.0000
Epoch 4435/50000
9/9 [==============================] - ETA: 0s

KeyboardInterrupt: 

<h2>保存模型</h2>

In [ ]:
save_model(model=model, filepath="../models/mining-dector/")

<h2>加载模型</h2>

In [ ]:
# model = load_model(filepath="../models/mining-dector/", custom_objects={"add_features": add_features})

<h1>预测测试集</h1>

<h2>处理数据</h2>

<h3>处理档案数据</h3>

In [ ]:
cons_info_test = pd.read_csv("../data/测试组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info_test

In [ ]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

In [ ]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

In [ ]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

In [ ]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

In [ ]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

In [ ]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

In [ ]:
cons_info_test.info()

In [ ]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

In [ ]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

In [ ]:
cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

In [ ]:
cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

In [ ]:
cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

In [ ]:
cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

In [ ]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test

In [ ]:
cons_info_test_values = cons_info_test.values
cons_info_test_values.shape

<h3>处理月电量数据</h3>

In [ ]:
elec_month_test = pd.read_csv("../data/测试组_比特币挖矿_月用电明细（20211217）.csv")
elec_month_test

In [ ]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test

elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]] = elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]] / elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]].max(axis=0)
elec_month_test

In [ ]:
cons_month_result_test = []
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_month_result_test.append(df.values)
cons_month_test = np.array(cons_month_result_test)
cons_month_test.shape

<h3>处理日电量数据</h3>

In [ ]:
elec_day_test = pd.read_csv("../data/测试组_比特币挖矿_日用电明细（20211217）.csv")
elec_day_test

elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] = elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] / elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]].max(axis=0)
elec_day_test

In [ ]:
dates = pd.to_datetime(elec_day_test["rq"].unique()).sort_values()

In [ ]:
cons_day_result_test = []
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id].drop(columns=["id"]).sort_values("rq")
    df["rq"] = pd.to_datetime(df["rq"])
    df = df.set_index("rq").reindex(dates)
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_day_result_test.append(df.values)
cons_day_test = np.array(cons_day_result_test)
cons_day_test.shape

<h2>预测数据</h2>

In [ ]:
y_pred = model.predict([cons_info_test_values, cons_month_test, cons_day_test])
y_pred

In [ ]:
cons_info_test["label"] = (y_pred > 0.5).astype(int)
cons_info_test.index.name = "id"
cons_info_test

In [ ]:
cons_info_test[["label"]].to_csv("../data/result20220211-3.csv", index=True)